In [8]:
# Loads gaze-related metrics for each trial and participant and performs statistical analysis for the whole study 

In [36]:
import re
import matplotlib.pyplot as plt
from enum import Enum
import os
import statistics
import numpy as np
import get_tcm as tcm
import csv
import pandas as pd

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg

from statsmodels.stats.anova import AnovaRM

from scipy.stats import sem, t, ttest_rel, shapiro, wilcoxon, mannwhitneyu


In [3]:
data_dir="Z:/builds/aplause-mr/UNITY_APP/aplause-mr-20240707-studystate-v1/gazeAnalysis"

file_path = data_dir + "/gaze_classification_analysis_results.csv"

df = pd.read_csv(file_path)
df

,participant_id,trial,group,condition,audio_condition,avatar_condition,gaze_local_pc,gaze_remote_adjacent_pc,gaze_remote_opposite_pc,gaze_others_pc,...,glance_remote_opposite_per_min,mutual_gaze_local_pc,mutual_gaze_local_mean_duration,mutual_gaze_local_per_min,mutual_gaze_remote_opp_pc,mutual_gaze_remote_opp_mean_duration,mutual_gaze_remote_opp_per_min,mutual_gaze_remote_adj_pc,mutual_gaze_remote_adj_mean_duration,mutual_gaze_remote_adj_per_min
0,0,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,18.387516,12.327698,55.786736,86.501951,...,12.795839,2.600780,1.111111,1.404421,37.295189,1.541935,14.512354,4.005202,0.855556,2.808843
1,1,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,5.331599,22.288687,39.817945,67.438231,...,11.235371,2.600780,1.111111,1.404421,12.145644,0.676812,10.767230,2.782835,0.972727,1.716515
2,2,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,3.172952,14.668401,58.179454,76.020806,...,10.299090,1.014304,0.780000,0.780234,37.295189,1.541935,14.512354,2.782835,0.972727,1.716515
3,3,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,13.888166,30.195059,33.211964,77.295189,...,15.136541,1.014304,0.780000,0.780234,12.145644,0.676812,10.767230,4.005202,0.855556,2.808843
4,0,1,0,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,17.973062,5.012144,66.173548,89.158755,...,9.406050,0.419519,0.475000,0.529918,40.759550,1.619298,15.102672,1.368956,0.885714,0.927357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,3,2,19,SPATIAL_VOLUMETRIC,SPATIAL,VOLUMETRIC,8.477041,19.925203,40.556825,68.959069,...,17.826719,0.000000,-1.000000,0.000000,21.961355,0.794737,16.580096,2.888012,0.695000,2.493247
252,0,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,28.726356,6.027743,22.723834,57.477932,...,8.776797,7.717528,0.927273,4.993695,4.590164,0.791304,3.480454,1.488020,0.536364,1.664565
253,1,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,24.262295,3.051702,46.506936,73.820933,...,18.612863,7.717528,0.927273,4.993695,16.973518,0.701042,14.527112,0.151324,0.600000,0.151324
254,2,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,2.118537,5.346784,18.486759,25.952081,...,11.803279,0.075662,0.150000,0.302648,4.590164,0.791304,3.480454,0.151324,0.600000,0.151324


In [56]:
# outlier / missing data detection

# Select numerical and text columns
numerical_cols = df.select_dtypes(include='number').columns.drop(['trial', 'participant_id', 'group'])
text_cols = ['condition', 'audio_condition', 'avatar_condition']

negative_rows = df[df[numerical_cols].lt(0).any(axis=1)]
negative_rows

,participant_id,trial,group,condition,audio_condition,avatar_condition,gaze_local_pc,gaze_remote_adjacent_pc,gaze_remote_opposite_pc,gaze_others_pc,...,glance_remote_opposite_per_min,mutual_gaze_local_pc,mutual_gaze_local_mean_duration,mutual_gaze_local_per_min,mutual_gaze_remote_opp_pc,mutual_gaze_remote_opp_mean_duration,mutual_gaze_remote_opp_per_min,mutual_gaze_remote_adj_pc,mutual_gaze_remote_adj_mean_duration,mutual_gaze_remote_adj_per_min
36,0,1,2,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT,10.066329,2.926258,42.684354,55.676941,...,15.216543,0.000000,-1.000000,0.000000,3.277409,0.600000,3.277409,0.507218,1.300000,0.234101
37,1,1,2,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT,1.950839,27.155677,31.642606,60.749122,...,11.236832,0.000000,-1.000000,0.000000,24.346469,1.273469,11.470933,4.330862,0.740000,3.511510
40,0,2,2,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,5.732123,0.510783,51.049943,57.292849,...,16.004540,0.000000,-1.000000,0.000000,3.093076,0.726667,2.553916,0.000000,-1.000000,0.000000
41,1,2,2,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,0.425653,32.604994,22.956867,55.987514,...,8.683314,0.000000,-1.000000,0.000000,19.920545,1.462500,8.172531,1.759364,0.442857,2.383655
43,3,2,2,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,17.026107,1.333712,71.935301,90.295119,...,6.640182,0.766175,0.675000,0.681044,19.920545,1.462500,8.172531,0.000000,-1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,3,2,16,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT,10.396040,18.637158,34.449621,63.482819,...,6.988934,2.184042,0.833333,1.572510,22.218987,1.338596,9.959231,0.000000,-1.000000,0.000000
242,2,0,19,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT,3.116559,8.456264,18.553916,30.126740,...,11.718263,0.000000,-1.000000,0.000000,5.402036,0.666667,4.861833,0.353210,0.425000,0.498649
243,3,0,19,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT,8.560150,13.567422,47.080823,69.208394,...,20.569292,0.000000,-1.000000,0.000000,31.830459,1.014570,18.824018,0.041554,0.100000,0.249325
250,2,2,19,SPATIAL_VOLUMETRIC,SPATIAL,VOLUMETRIC,1.786827,10.887181,20.299190,32.973198,...,9.723665,0.000000,-1.000000,0.000000,5.879909,0.764865,4.612508,1.205070,0.644444,1.121961


In [14]:
# Create dataset that averages numeric data from each trial



# specify that numeric cols should be averaged, and non-numeric should just take first value 
agg_dict = {col: 'mean' for col in numerical_cols}  
agg_dict.update({col: 'first' for col in text_cols})

averaged_trial_df = df.groupby(['trial', 'group']).agg(agg_dict).reset_index()
averaged_trial_df

,trial,group,gaze_local_pc,gaze_remote_adjacent_pc,gaze_remote_opposite_pc,gaze_others_pc,gaze_transition_pc,gaze_avert_pc,pc_all,gaze_transition_mean_duration_sec,...,mutual_gaze_local_per_min,mutual_gaze_remote_opp_pc,mutual_gaze_remote_opp_mean_duration,mutual_gaze_remote_opp_per_min,mutual_gaze_remote_adj_pc,mutual_gaze_remote_adj_mean_duration,mutual_gaze_remote_adj_per_min,condition,audio_condition,avatar_condition
0,0,0,10.195059,19.869961,46.749025,76.814044,7.158648,15.812744,99.785436,0.335662,...,1.092328,24.720416,1.109374,12.639792,3.394018,0.914141,2.262679,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC
1,0,1,21.158129,9.014477,31.252784,61.425390,9.816258,28.641425,99.883073,0.328170,...,2.939866,10.356347,0.685544,8.685969,1.013363,0.466667,1.002227,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT
2,0,2,6.485627,14.585297,39.862158,60.933082,9.419180,29.241282,99.593544,0.319906,...,0.494816,12.912347,1.063651,6.573987,3.169180,0.548485,2.474081,SPATIAL_VOLUMETRIC,SPATIAL,VOLUMETRIC
3,0,3,8.303134,8.265828,41.979322,58.548284,8.654871,32.711575,99.914730,0.290101,...,1.151140,15.668301,0.639328,11.191644,1.577489,0.240000,0.639522,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT
4,0,4,7.929422,23.426871,40.252976,71.609269,8.407738,19.483418,99.500425,0.328478,...,0.127551,16.326531,1.291352,7.525510,8.588435,1.096581,4.464286,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3,13,12.273476,5.546403,44.371225,62.191104,11.669412,25.233388,99.093904,0.316912,...,1.153213,16.913784,0.640208,15.156507,0.109830,-0.300000,0.164745,SPATIAL_ABSTRACT,SPATIAL,ABSTRACT
60,3,14,15.072922,13.009081,43.209961,71.291965,14.164832,14.522565,99.979362,0.348488,...,2.228949,21.271326,0.818225,14.611998,0.990644,0.450000,1.320859,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC
61,3,15,9.210910,10.545508,38.674154,58.430572,13.032380,28.376604,99.839557,0.324212,...,1.575263,13.681447,0.660992,12.427071,0.466744,0.413333,0.700117,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT
62,3,16,8.365335,9.760625,43.740399,61.866359,8.986175,28.833845,99.686380,0.306534,...,0.768049,19.828469,0.864266,13.440860,0.678443,0.755000,0.537634,SPATIAL_VOLUMETRIC,SPATIAL,VOLUMETRIC


In [39]:

# to get started: effect of conditions on gaze_local_pc?

# check differences in mean by condition
pd.DataFrame(round(averaged_trial_df.groupby(['audio_condition'])['gaze_local_pc'].mean(),2)).reset_index()
# pd.DataFrame(round(averaged_trial_df.groupby(['avatar_condition'])['gaze_local_pc'].mean(),2)).reset_index()
# pd.DataFrame(round(averaged_trial_df.groupby(['avatar_condition', 'audio_condition'])['gaze_local_pc'].mean(),2)).reset_index()

# TODO: check if ANOVA is valid here (e.g. check for normality / sphericity)


# between correct?
# model2 = pg.anova(dv='gaze_local_pc', between=['avatar_condition', 'audio_condition'], data=averaged_trial_df, detailed=True)
# round(model2, 4)


# visalize the results for this dependent variable

# Fit the model using OLS
# model = ols('gaze_local_pc ~ C(audio_condition) * C(avatar_condition)', data=averaged_trial_df).fit()

# Perform two-way ANOVA
# anova_table = sm.stats.anova_lm(model, typ=2)

# # Display the results
# print(anova_table)

,audio_condition,gaze_local_pc
0,DIOTIC,10.53
1,SPATIAL,9.89


In [46]:
def RunAnova(variable):
    verbose=True

    anova_rm = AnovaRM(data=averaged_trial_df, depvar=variable, subject='group', within=['audio_condition', 'avatar_condition'])
    res = anova_rm.fit()
    # res_dict = {q: res}

    # check for normal distribution
    nres = shapiro(averaged_trial_df[variable])

    if verbose:
    #     print(q+': '+get_q(df, q))
        print('Variable: ' + variable)
        print('Normality is assumed' if nres.pvalue>0.05 else 'Not normally distributed')
        print(res.anova_table)
        print('\n')

RunAnova('gaze_local_pc')

Variable: gaze_local_pc
Normality is assumed
                                   F Value  Num DF  Den DF    Pr > F
audio_condition                   4.851113     1.0    15.0  0.043685
avatar_condition                  1.742889     1.0    15.0  0.206565
audio_condition:avatar_condition  3.977253     1.0    15.0  0.064623




In [50]:
non_mutual_numeric_cols = numerical_cols.drop([c for c in numerical_cols if "mutual" in c])

for col in non_mutual_numeric_cols:
    RunAnova(col)


Variable: gaze_local_pc
Normality is assumed
                                   F Value  Num DF  Den DF    Pr > F
audio_condition                   4.851113     1.0    15.0  0.043685
avatar_condition                  1.742889     1.0    15.0  0.206565
audio_condition:avatar_condition  3.977253     1.0    15.0  0.064623


Variable: gaze_remote_adjacent_pc
Normality is assumed
                                    F Value  Num DF  Den DF    Pr > F
audio_condition                    0.604154     1.0    15.0  0.449083
avatar_condition                  36.519041     1.0    15.0  0.000023
audio_condition:avatar_condition   0.977902     1.0    15.0  0.338402


Variable: gaze_remote_opposite_pc
Normality is assumed
                                   F Value  Num DF  Den DF    Pr > F
audio_condition                   0.017109     1.0    15.0  0.897671
avatar_condition                  4.189596     1.0    15.0  0.058611
audio_condition:avatar_condition  0.001061     1.0    15.0  0.974439


Variabl